In [69]:
from math import radians, cos, sin, asin, sqrt
import pandas as pd
import numpy as np

# Import, lowercase city/state name to attempt join later

# Geonames is ~1:1 City/Place names to Zip Codes
# Schema: http://download.geonames.org/export/zip/readme.txt
col_names = ['country_code',
             'postal_code',
             'place_name',
             'admin_name1',
             'admin_code1',
             'admin_name2',
             'admin_code2',
             'admin_name3',
             'admin_code3',
             'latitude',
             'longitude',
             'accuracy']
geonames = pd.read_table('GeoNamesZip.txt', header=None, names=col_names, dtype={'postal_code': str})
geonames['place_name'] = geonames['place_name'].str.lower()
geonames['admin_code1'] = geonames['admin_code1'].str.lower()
geonames_set = set(geonames['postal_code'])

# Free-zip 1 is 1:1 City to Zip Code
free_zip_db = pd.read_csv('free-zipcode-database-Primary.csv', dtype={'Zipcode': str})
free_zip_db['City'] = free_zip_db['City'].str.lower()
free_zip_db['State'] = free_zip_db['State'].str.lower()
freezip_set = set(free_zip_db['Zipcode'])

# Free-zip all is Many:1 City to Zip Code
free_zip_all = pd.read_csv('free-zipcode-database-all-places.csv', dtype={'Zipcode': str})
# We only want Primary/Acceptable locs. There is some junk here.
free_zip_all = free_zip_all[free_zip_all['LocationType'].isin(['ACCEPTABLE', 'PRIMARY'])]
free_zip_all['City'] = free_zip_all['City'].str.lower()
free_zip_all['State'] = free_zip_all['State'].str.lower()
freezip_all_set = set(free_zip_all['Zipcode'])

#
noncensus = pd.read_csv('noncensus_zip.csv', dtype={'zip': str})
noncensus['city'] = noncensus['city'].str.lower()
noncensus['state'] = noncensus['state'].str.lower()
noncensus_set = set(noncensus['zip'])

set_lengths = {
    'noncensus_set': len(noncensus_set),
    'free_zip_set': len(freezip_set),
    'free_zip_all_set': len(freezip_all_set),
    'geonames_set': len(geonames_set)
    }
set_lengths

{'free_zip_all_set': 42522,
 'free_zip_set': 42522,
 'geonames_set': 43586,
 'noncensus_set': 43524}

In [70]:
len(geonames), len(free_zip_db), len(free_zip_all), len(noncensus)

(43629, 42522, 56725, 43524)

In [151]:
# Columns of interest, rename for specificity
geo_zip_place_loc = geonames[['postal_code', 'place_name', 'admin_code1',
                              'latitude', 'longitude']]
geo_zip_place_loc = geo_zip_place_loc.rename(columns={'postal_code': 'Zip_geo',
                                                      'latitude': 'Lat_geo', 
                                                      'longitude': 'Lon_geo',
                                                      'place_name': 'City_geo',
                                                      'admin_code1': 'State_geo'})

free_zip_all_place_loc = free_zip_all[['Zipcode', 'City', 'State', 'Lat', 'Long']]
free_zip_all_place_loc = free_zip_all_place_loc.rename(columns={'Zipcode': 'Zip_free',
                                                                'Lat': 'Lat_free',
                                                                'Long': 'Lon_free',
                                                                'City': 'City_free',
                                                                'State': 'State_free'})
noncensus_place_loc = noncensus[['zip', 'city', 'state', 'latitude', 'longitude']]
noncensus_place_loc = noncensus_place_loc.rename(columns={'zip': 'Zip_non',
                                                          'latitude': 'Lat_non', 
                                                          'longitude': 'Lon_non',
                                                          'city': 'City_non',
                                                          'state': 'State_non'})

In [152]:
# free-zip vs geonames
merged_zip = pd.merge(free_zip_all_place_loc, geo_zip_place_loc, 
                      left_on='Zip_free', right_on='Zip_geo')
merged_zip_city = pd.merge(free_zip_all_place_loc, geo_zip_place_loc, 
                           left_on=['Zip_free', 'City_free'], 
                           right_on=['Zip_geo', 'City_geo'])
merged_zip_state = pd.merge(free_zip_all_place_loc, geo_zip_place_loc, 
                            left_on=['Zip_free', 'State_free'], 
                            right_on=['Zip_geo', 'State_geo'])
merged_zip_city_state = pd.merge(free_zip_all_place_loc, geo_zip_place_loc, 
                                 left_on=['Zip_free', 'City_free', 'State_free'], 
                                 right_on=['Zip_geo', 'City_geo', 'State_geo'])
lengths_free_geo = {
    'merged_zip': len(merged_zip),
    'merged_zip_city': len(merged_zip_city),
    'merged_zip_state': len(merged_zip_state),
    'merged_zip_city_state': len(merged_zip_city_state)
}
lengths_free_geo

{'merged_zip': 55777,
 'merged_zip_city': 41386,
 'merged_zip_city_state': 41386,
 'merged_zip_state': 55777}

In [88]:
merged_zip

,Zip_free,City_free,State_free,Lat_free,Lon_free,Zip_geo,City_geo,State_geo,Lat_geo,Lon_geo
0,07675,westwood,nj,40.98,-74.03,07675,westwood,nj,41.0092,-74.0041
1,07675,old tappan,nj,40.98,-74.03,07675,westwood,nj,41.0092,-74.0041
2,07675,river vale,nj,40.98,-74.03,07675,westwood,nj,41.0092,-74.0041
3,07675,rivervale,nj,40.98,-74.03,07675,westwood,nj,41.0092,-74.0041
4,07677,woodcliff lake,nj,41.02,-74.05,07677,woodcliff lake,nj,41.0234,-74.0603
5,07677,westwood,nj,41.02,-74.05,07677,woodcliff lake,nj,41.0234,-74.0603
6,07677,woodcliff lk,nj,41.02,-74.05,07677,woodcliff lake,nj,41.0234,-74.0603
7,07885,wharton,nj,40.89,-74.58,07885,wharton,nj,40.9139,-74.5863
8,07981,whippany,nj,40.82,-74.41,07981,whippany,nj,40.8219,-74.4200
9,07999,whippany,nj,40.82,-74.41,07999,whippany,nj,40.8673,-74.5783


In [153]:
# noncensus vs free zip
merged_non_free_zip = pd.merge(free_zip_all_place_loc, noncensus_place_loc,
                               left_on=['Zip_free'], right_on=['Zip_non'])
merged_non_free_zip_city = pd.merge(free_zip_all_place_loc, noncensus_place_loc,
                                    left_on=['Zip_free', 'City_free'], right_on=['Zip_non', 'City_non'])
merged_non_free_zip_state = pd.merge(free_zip_all_place_loc, noncensus_place_loc,
                                     left_on=['Zip_free', 'State_free'], right_on=['Zip_non', 'State_non'])
merged_non_free_zip_city_state = pd.merge(free_zip_all_place_loc, noncensus_place_loc,
                                          left_on=['Zip_free', 'City_free', 'State_free'], 
                                          right_on=['Zip_non', 'City_non', 'State_non'])
lengths_noncensus_free = {
    'merged_zip': len(merged_non_free_zip),
    'merged_zip_city': len(merged_non_free_zip_city),
    'merged_zip_state': len(merged_non_free_zip_state),
    'merged_zip_city_state': len(merged_non_free_zip_city_state)
}
lengths_noncensus_free

{'merged_zip': 55867,
 'merged_zip_city': 41461,
 'merged_zip_city_state': 41435,
 'merged_zip_state': 55829}

In [154]:
merged_non_free_zip

,Zip_free,City_free,State_free,Lat_free,Lon_free,Zip_non,City_non,State_non,Lat_non,Lon_non
0,00705,aibonito,pr,18.14,-66.26,00705,aibonito,pr,18.129420,-66.265410
1,00610,anasco,pr,18.28,-67.14,00610,anasco,pr,18.288319,-67.136040
2,00611,angeles,pr,18.28,-66.79,00611,angeles,pr,18.279531,-66.802170
3,00612,arecibo,pr,18.45,-66.73,00612,arecibo,pr,18.449732,-66.698790
4,00601,adjuntas,pr,18.16,-66.72,00601,adjuntas,pr,18.180103,-66.749470
5,00631,castaner,pr,18.19,-66.82,00631,castaner,pr,18.186739,-66.851740
6,00631,adjuntas,pr,18.19,-66.82,00631,castaner,pr,18.186739,-66.851740
7,00602,aguada,pr,18.38,-67.18,00602,aguada,pr,18.363285,-67.180240
8,00603,aguadilla,pr,18.43,-67.15,00603,aguadilla,pr,18.448619,-67.134220
9,00603,ramey,pr,18.43,-67.15,00603,aguadilla,pr,18.448619,-67.134220


In [155]:
# What are our state conflicts?
merged_non_free_zip[~merged_non_free_zip['Zip_free'].isin(merged_non_free_zip_state['Zip_free'])]

,Zip_free,City_free,State_free,Lat_free,Lon_free,Zip_non,City_non,State_non,Lat_non,Lon_non
60,10004,new york,ny,40.71,-73.99,10004,new york,nj,40.699226,-74.041180
61,10004,bowling green,ny,40.71,-73.99,10004,new york,nj,40.699226,-74.041180
12741,38041,henning,tn,35.67,-89.57,38041,henning,ar,35.629555,-89.869450
12742,38041,fort pillow,tn,35.67,-89.57,38041,henning,ar,35.629555,-89.869450
13161,38063,ripley,tn,35.74,-89.53,38063,ripley,ar,35.637993,-89.868590
13274,38079,tiptonville,tn,36.37,-89.47,38079,tiptonville,ky,36.513386,-89.504720
18012,52761,muscatine,ia,41.41,-91.07,52761,muscatine,il,41.413372,-91.006120
18629,56129,ellsworth,mn,43.52,-96.01,56129,ellsworth,ia,43.495384,-95.906910
18630,56027,elmore,mn,43.50,-94.08,56027,elmore,ia,43.482104,-94.098370
18890,55954,mabel,mn,43.52,-91.76,55954,mabel,ia,43.498082,-91.894290


In [158]:
# Merge all the things
merged_all = pd.merge(merged_non_free_zip, merged_zip,
                      how='left',
                      left_on=['Zip_free', 'City_free', 'State_free'], 
                      right_on=['Zip_free', 'City_free', 'State_free'])
merged_all

,Zip_free,City_free,State_free,Lat_free_x,Lon_free_x,Zip_non,City_non,State_non,Lat_non,Lon_non,Lat_free_y,Lon_free_y,Zip_geo,City_geo,State_geo,Lat_geo,Lon_geo
0,00705,aibonito,pr,18.14,-66.26,00705,aibonito,pr,18.129420,-66.265410,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00610,anasco,pr,18.28,-67.14,00610,anasco,pr,18.288319,-67.136040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00611,angeles,pr,18.28,-66.79,00611,angeles,pr,18.279531,-66.802170,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00612,arecibo,pr,18.45,-66.73,00612,arecibo,pr,18.449732,-66.698790,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00601,adjuntas,pr,18.16,-66.72,00601,adjuntas,pr,18.180103,-66.749470,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,00631,castaner,pr,18.19,-66.82,00631,castaner,pr,18.186739,-66.851740,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,00631,adjuntas,pr,18.19,-66.82,00631,castaner,pr,18.186739,-66.851740,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,00602,aguada,pr,18.38,-67.18,00602,aguada,pr,18.363285,-67.180240,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,00603,aguadilla,pr,18.43,-67.15,00603,aguadilla,pr,18.448619,-67.134220,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,00603,ramey,pr,18.43,-67.15,00603,aguadilla,pr,18.448619,-67.134220,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
master_zips = merged_all[['Zip_free', 'Zip_geo', 'Zip_non',
                          'City_free', 'City_geo', 'City_non',
                          'State_free', 'State_geo', 'State_non',
                          'Lat_free_x', 'Lat_geo', 'Lat_non',
                          'Lon_free_x', 'Lon_geo', 'Lon_non']]
master_zips = master_zips.rename(columns={'Lon_free_x': 'Lon_free', 
                                          'Lat_free_x': 'Lat_free'})

In [163]:
# Let's check the max Haversine distance between points
def check_haversine(row):
    """
    Great circle distance between our Lat/Lon points (dec degrees)
    """
    # Convert to Radians
    rad_lat_1, rad_lon_1 = map(radians, [row['Lat_free'], row['Lon_free']])
    rad_lat_2, rad_lon_2 = map(radians, [row['Lat_geo'], row['Lon_geo']])
    rad_lat_3, rad_lon_3 = map(radians, [row['Lat_non'], row['Lon_non']])
    
    # 1 vs 2, 1 vs 3, 2 vs 3
    pairs = [(rad_lat_1, rad_lon_1, rad_lat_2, rad_lon_2),
             (rad_lat_1, rad_lon_1, rad_lat_3, rad_lon_3),
             (rad_lat_2, rad_lon_2, rad_lat_3, rad_lon_3)]
    
    # Only get pairs that don't have nan vals
    filtered_pairs = filter(lambda x: not any(map(np.isnan, x)), pairs)

    def get_dist(lat_1, lon_1, lat_2, lon_2):
        under_root = (sin((lat_2 - lat_1)/2)**2 + 
                      cos(lat_1) * cos(lat_2) * sin((lon_2 - lon_1)/2)**2)
        # 6367 = Earth radius in kilometers
        return 6367 * 2 * asin(sqrt(under_root)) 
                      
    row['Max_Haversine_Dist'] = max([get_dist(*pair) for pair in filtered_pairs])
    row['Min_Haversine_Dist'] = min([get_dist(*pair) for pair in filtered_pairs])
    return row

master_zips = master_zips.apply(check_haversine, axis=1)

In [164]:
master_zips.sort(['Min_Haversine_Dist', 'Max_Haversine_Dist'], ascending=False)

,Zip_free,Zip_geo,Zip_non,City_free,City_geo,City_non,State_free,State_geo,State_non,Lat_free,Lat_geo,Lat_non,Lon_free,Lon_geo,Lon_non,Max_Haversine_Dist,Min_Haversine_Dist
141,00785,NaN,00785,guayama,NaN,guayama,pr,NaN,pr,17.97,NaN,18.018822,-66.11,NaN,-66.795603,72.663892,72.663892
338,00765,NaN,00765,vieques,NaN,vieques,pr,NaN,pr,18.42,NaN,18.125664,-65.83,NaN,-65.456030,51.254825,51.254825
29824,99726,99726,99726,bettles field,bettles field,bettles field,ak,ak,ak,67.24,65.2264,66.917381,-152.27,-151.0251,-151.505080,230.591072,48.797534
29994,99754,99754,99754,koyukuk,koyukuk,koyukuk,ak,ak,ak,65.04,65.2264,64.881745,-158.66,-151.0251,-157.704390,357.154565,48.262018
29874,99666,99666,99666,nunam iqua,sheldon point,sheldon point,ak,ak,ak,62.09,62.1172,62.495675,-165.18,-163.2376,-164.971880,101.031829,46.345352
29838,99566,99566,99566,chitina,chitina,chitina,ak,ak,ak,61.45,61.4710,61.555701,-143.15,-144.9910,-144.142330,97.766304,45.954722
27538,87018,87018,87018,counselor,counselor,counselor,nm,nm,nm,36.13,35.7174,36.203634,-106.93,-106.9358,-107.494410,73.784099,45.853192
30050,99653,99653,99653,port alsworth,port alsworth,port alsworth,ak,ak,ak,60.46,58.2687,60.102331,-154.19,-156.6484,-154.557080,280.448546,44.594529
31165,92328,92328,92328,death valley,death valley,death valley,ca,ca,ca,36.29,36.4672,35.945641,-116.45,-116.8937,-117.202960,77.671735,44.312650
29797,99557,99557,99557,aniak,aniak,aniak,ak,ak,ak,61.20,60.3147,61.570981,-158.60,-163.1189,-158.880720,267.901140,43.848750


In [166]:
# Get Primary City and State based on agreement
def get_primary(r):
    cities = [r['City_free'], r['City_geo'], r['City_non']]
    states = [r['State_free'], r['State_geo'], r['State_non']]
    
    def get_most_common(lst):
        cet = set(lst)
        write_val = None
        if len(cet) == 1:
            # If they all agree, use that value
            write_val = cet.pop()
        elif len(cet) == 2:
            # If 2/3 agree, go with it
            write_val = max(cet, key=lst.count)
        else:
            # If None agree, good to know
            write_val = None
        return write_val
    
    r['Primary_City'] = get_most_common(cities)
    r['Primary_State'] = get_most_common(states)
    return r
master_zips = master_zips.apply(get_primary, axis=1)

In [167]:
master_zips[master_zips['Primary_City'].isin([None])]

,Zip_free,Zip_geo,Zip_non,City_free,City_geo,City_non,State_free,State_geo,State_non,Lat_free,Lat_geo,Lat_non,Lon_free,Lon_geo,Lon_non,Max_Haversine_Dist,Min_Haversine_Dist,Primary_City,Primary_State
6,00631,NaN,00631,adjuntas,NaN,castaner,pr,NaN,pr,18.19,NaN,18.186739,-66.82,NaN,-66.851740,3.370418,3.370418,None,pr
9,00603,NaN,00603,ramey,NaN,aguadilla,pr,NaN,pr,18.43,NaN,18.448619,-67.15,NaN,-67.134220,2.654852,2.654852,None,pr
11,00604,NaN,00604,ramey,NaN,aguadilla,pr,NaN,pr,18.43,NaN,18.498987,-67.15,NaN,-67.136990,7.787871,7.787871,None,pr
58,10002,10002,10002,knickerbocker,new york city,new york,ny,ny,ny,40.71,40.7152,40.717040,-73.99,-73.9877,-73.987000,0.822119,0.212801,None,ny
61,10004,10004,10004,bowling green,new york city,new york,ny,ny,nj,40.71,40.6964,40.699226,-73.99,-74.0253,-74.041180,4.474649,1.374261,None,ny
63,10005,10005,10005,wall street,new york city,new york,ny,ny,ny,40.71,40.7056,40.706019,-73.99,-74.0083,-74.008580,1.626455,0.052195,None,ny
65,10006,10006,10006,trinity,new york city,new york,ny,ny,ny,40.71,40.7085,40.707904,-73.99,-74.0135,-74.013420,1.986553,0.066573,None,ny
72,10012,10012,10012,prince,new york city,new york,ny,ny,ny,40.71,40.7255,40.725960,-73.99,-73.9983,-73.998340,1.907596,0.051228,None,ny
74,10013,10013,10013,canal street,new york city,new york,ny,ny,ny,40.71,40.7185,40.720666,-73.99,-74.0025,-74.005260,1.748400,0.334620,None,ny
75,10013,10013,10013,chinatown,new york city,new york,ny,ny,ny,40.71,40.7185,40.720666,-73.99,-74.0025,-74.005260,1.748400,0.334620,None,ny


In [36]:
# Let's see if we can get better geo accuracy. Using TwoFishes geocoder and DataScienceToolkit
def get_forward(row):
    twofishes_url = 'http://localhost:8081/search/geocode'
    datasci_url = 'http://www.datasciencetoolkit.org/maps/api/geocode/json'
    
    tf_resp = requests.get(twofishes_url, params={'query': '{},+{}